In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.model_selection import cross_val_score

In [3]:
df= pd.read_csv("train_preprocessed.csv",index_col = 0)
#df_test = pd.read_csv("test_preprocessed.csv",index_col = 0)

In [4]:
df.head()

,Survived,Pclass,Sex,Embarked,ischild,hasCabin,Hasfam
PassengerId,,,,,,,
1,0,3,0,1,0,0,1
2,1,1,1,1,0,1,1
3,1,3,1,1,0,0,0
4,1,1,1,1,0,1,1
5,0,3,0,1,0,0,0


In [5]:
X = df.drop(["Survived"],axis=1)

In [6]:
y =df["Survived"]

In [7]:
X.head()

,Pclass,Sex,Embarked,ischild,hasCabin,Hasfam
PassengerId,,,,,,
1,3,0,1,0,0,1
2,1,1,1,0,1,1
3,3,1,1,0,0,0
4,1,1,1,0,1,1
5,3,0,1,0,0,0


In [8]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [9]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.1)

In [10]:
neigh =  KNeighborsClassifier()
lin= LogisticRegression() 
vect= SVC()
tree = DecisionTreeClassifier()

In [59]:
lname=["neighbors","linear","vector","tree"]
#lname=["neighbors","vector","tree"]

In [60]:
lclf= [neigh,lin,vect,tree]
#lclf= [neigh,vect,tree]

In [13]:
# 4 grilles d'hyperparamètres

In [61]:
param_list = [{"n_neighbors": [2,3,5,7,9], "weight": ['uniform', 'distance']},
              {"solver" : ['newton-cg', 'lbfgs', 'liblinear']},
              {"kernel":['linear', 'poly', 'rbf', 'sigmoid'],'C': [0.1, 1, 10, 100],'gamma':['scale', 'auto']},
              {'criterion' : ["gini", "entropy"],'splitter' :["best", "random"]}]


In [62]:
gridcvs = {}

In [63]:
for Name,Clf,Param in zip(lname,lclf,param_list):
    gridcvs[Name] = GridSearchCV(Clf,Param)
    #print(Name,Clf,Param)
    

In [64]:
strat = StratifiedKFold(n_splits=5)

In [65]:
outerscore = {}

In [66]:
for name,gcv in gridcvs.items():
    nested_score = cross_val_score(gcv,X_train,y_train,cv=strat)
    outerscore[name] = nested_score
    print(f"{name}: outer accuracy {100 * nested_score.mean():.2f} +/- { 100 * nested_score.std():.2f}")

C:\ProgramData\miniforge3\envs\my-env\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\miniforge3\envs\my-env\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\miniforge3\envs\my-env\lib\site-packages\sklearn\model_selection\_search.py", line 891, in fit
    self._run_search(evaluate_candidates)
  File "C:\ProgramData\miniforge3\envs\my-env\lib\site-packages\sklearn\model_selection\_search.py", line 1392, in _run_search


neighbors: outer accuracy nan +/- nan
linear: outer accuracy 78.90 +/- 1.58
vector: outer accuracy 81.77 +/- 2.21
tree: outer accuracy 81.90 +/- 2.09


In [73]:
#selection du meilleur modèle
